In [1]:
# import nest_asyncio
# nest_asyncio.apply()
import os
import django
import inspect

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from books.models import Book
from books import utils

from asgiref.sync import sync_to_async
from django.shortcuts import get_object_or_404
from django.http import HttpRequest

In [2]:
def get_sql_queryparams(model, lookup_dict: dict, delim='AND') -> str:
    """
    accepts a dictionary of kwargs and returns a string contains query
    parameters.
    example:
    get_sql_queryparams(Book,{'col1':'something', 'col2':'other_thing'})
    >>> 'books_book.col1=$$something$$ AND books_book.col2=$$other_thing$$ '
    """
    table = model._meta.db_table
    q = [f"{table}.{i}=%$${j}$$% " for (i, j) in lookup_dict.items()]
    length = len(q)
    for i in range(length):
        if i < length - 1:
            q[i] += delim+' '
    return ''.join(q)
request = {'title':'d467227cab2b40a8','serial_number':"sdf"}
str(Book.objects.get_queryset().query)
get_sql_queryparams(Book, request)

'books_book.title=%$$d467227cab2b40a8$$% AND books_book.serial_number=%$$sdf$$% '

In [4]:
# from django.db import connection
# connection.settings_dict['PASSWORD'] = '1234'
dir(HttpRequest().POST)
'd467227cab2b40a8' in request.values()

True

In [7]:
b = Book.objects.raw("select * from books_book as t where title=$$%s$$" % '11ef11b1d80aa8e0')
# from django.core.serializers import serialize
# serialize('json', b)


[]

In [9]:
# Book.objects.get(**{'title': 'd51804cb9cf0ff49', 'serial_number': 
#  'd51804cb9cf0ff49'})


In [36]:
lookup_fields = ['title', 'serial_number']
request = {'title':'d467227cab2b40a8','serial_number':"sdf"}
kwargs = {i:request[i] for i in lookup_fields if i in request}
kwargs


{'title': 'd467227cab2b40a8', 'serial_number': 'sdf'}

In [38]:
def get_sql_lookups(lookup_dict, lookup_fields):
    q = [f"table.{i}=$${j}$$" for (i, j) in lookup_dict.items()
         if i in lookup_fields]
    return ''.join(q)
get_sql_lookups(request)

'table.title=$$d467227cab2b40a8$$table.serial_number=$$sdf$$'

In [54]:
def get_sql_ordering(fields:dict[str, str]):

    args =  [f" {i} {fields[i]}" for i in fields]
    for i in range(len(args)-1):
        args[i] += ','
        
    return f'order by' + ''.join(args)

get_sql_ordering({'as':'ASC', 'sort_parameter':'DESC'})

[' as ASC', ' sort_parameter DESC']


'order by as ASC, sort_parameter DESC'

In [56]:
sql = 'SELECT * FROM books_book WHERE books_book.description ILIKE %s'


c = Book.objects.raw(sql,['%plant%'])
# dir(c)
# list(c)

In [91]:
L = ['a','b','c']
def get_ordering_enum(order_code):
    l = {}
    for i in order_code.split('.'):
        index = abs(int(i))
        if i.startswith('-'):
            print(abs(int(i)))
            
            l.update({L[index]:'DESC'})
        else:
            l.update({L[index]:'DESC'})
    return l


(get_ordering_enum('-2.1'))

2


{'c': 'DESC', 'b': 'DESC'}

In [12]:
dict( [('description', 'DESC'), ('serial_number', 'ASC')])

def get_sql_searchparams(model , search_fields, search_param, delim:str= 'AND') -> str:
    """

    """
    table = model._meta.db_table
    q = [f"UPPER({table}.{i}::text) ILIKE %s " for i in search_fields]
    length = len(q)
    for i in range(length):
        if i < length - 1:
            q[i] += delim+' '
    return ''.join(q)

get_sql_searchparams(Book, ['decription', 'blah'], 'plant')

'UPPER(books_book.decription::text) ILIKE %s AND UPPER(books_book.blah::text) ILIKE %s '

In [40]:
l = ['decription', 'blah']
d={'id': 'asc', 'title': 'desc', 'serial_number': 'desc'}
', '.join([f'{i} {d[i]}' for i in d])


'id asc, title desc, serial_number desc'

* = 585.92	

django = 
list_display : 